## Functional Programming

* refers to a programming paradigm where we treat a function as an object
* we define our logic using those function objects.
* it mostly based on two important function concepts
    1. we can pass a function to another as an argument.
    2. a function can return another function as an argument
    
#### Calculator Example


In [8]:
class Calculator:
    def present(self,value1, opr, value2):
        if opr == 'plus':
            result= value1 + value2
        elif opr == 'minus':
            result= value1 - value2
        elif opr == 'multiply':
            result= value1 * value2
        elif opr == 'divide':
            result= value1 / value2
        else:
            raise ValueError(f"Invalid operator:{opr}")

        print(f'{value1} {opr} {value2} = {result}')    
    

In [9]:
calc=Calculator()

calc.present(20, 'plus', 30)
calc.present(20, 'divide', 4)


20 plus 30 = 50
20 divide 4 = 5.0


In [10]:
calc.present(20, 'mod', 7)

ValueError: Invalid operator:mod

#### Problem

* we have a working calculator
* but the calculator is not extensible
* if we need more operations tomorrow (mod,power, permutation) we will have to modify the calculator again.
* similarly current calculator always displays the result on console
    * if tomorrow we want to display the result at different place (web page, GUI) this calcualtor will be useless, although it will still can peform the calculation

* this code has different responsibilities. mixed


#### Solution ---> Function Object

* treat each operation as function object
* pass the function object you want to use.
* also treat the print() function as one of the possible choices for presenting output.
* we can replace print with any other function of our choice.

#### Phase #1 create each operation as a separate function

In [11]:
def plus(a,b): return a+b
def minus(a,b): return a-b
def multiply(a,b): return a*b
def divide(a,b): return a/b

### Phase #2 calculator can take these (and more) functions as parameter

#### Approach #1
* user can pass the function that it wants to invoke

In [13]:
class Calculator:
    def present(self,value1, opr, value2):
        result=opr(value1,value2)
        print(f'{value1} {opr.__name__} {value2} = {result}') 

In [14]:
calc=Calculator()
calc.present(20, plus, 30)
calc.present(20, divide, 11)
# now you can't really pass a function, you don't have

20 plus 30 = 50
20 divide 11 = 1.8181818181818181


#### Problem: client may not be able to pass the function directly
* can we pass a string?

In [20]:
class Calculator:
    def __init__(self, presenter=None):
        self.presenter=presenter
        if self.presenter is None:
            self.presenter= lambda v1,opr,v2,result: print(f'{v1} {opr} {v2} => {result}')

        self._operators={}
        self._add_basic_operators()

    def _add_basic_operators(self):
        self.add_operator(plus)
        self.add_operator(minus)
        self.add_operator(multiply)
        self.add_operator(divide)


    def add_operator(self, operator, name=None):
        if name is None:
            name = operator.__name__
        self._operators[name.lower()]=operator

    def present(self,value1, opr, value2):
        opr=opr.lower()
        if opr in self._operators:
            fn=self._operators[opr]
            result=fn(value1,value2)
            print(f'{value1} {opr} {value2} = {result}')
        else:
            raise ValueError(f'No such operator: "{opr}"') 

In [21]:
calc=Calculator()

calc.present(20, 'plus', 30)

calc.present(20, 'Divide', 11)

20 plus 30 = 50
20 divide 11 = 1.8181818181818181


In [22]:
calc.present(20, 'mod',7)

ValueError: No such operator: "mod"

### But we can add mod logic dynamically

In [23]:
def mod(v1,v2): return v1%v2

calc.add_operator(mod)

In [24]:
calc.present(20, 'mod', 7)

20 mod 7 = 6


### we can also add functionality use lambda function

* but all lambdas are called lambda
* so we will a friendly name in add_operator function as optional second argument

In [26]:
calc.add_operator( lambda v1,v2: v1**v2, 'power')

In [27]:
calc.present(20, 'power', 3)

20 power 3 = 8000


#### Assignment 2.3

* write a search function that takes a sequence of values and returns
    * all even numbers
    * all prime numbers
    * all numbers divisible by 7
    * all permanent employees of an organization

* search should be your function
    * No built-in function use allowed.

### Solution #1

In [1]:
def search( values):
    result=[]
    for value in values:
        if value %2 ==0:
            result.append(value)
    return result

In [2]:
values=[2,9,11,8,4,13,3,72,8,4,25,33,27]



In [3]:
search(values)

[2, 8, 4, 72, 8, 4]

#### Best Practices

*  search shouldn't return a list.
    * it should yield.
    * never return a large list if it can be avoided.

In [4]:
def search( values):
    
    for value in values:
        if value %2 ==0:
            yield(value)
    

In [5]:
for evens in search(values):
    print(evens,  end=' ')

2 8 4 72 8 4 

### Problem --> it only searches evens.

* what about other requiremens like
    * search prime
    * search employees


### Approach #1

In [2]:
def search_evens( values):
    
    for value in values:
        if value %2 ==0:
            yield(value)

def is_prime(value):
    if value<2: return False
    for i in range(2,value):
        if value %i ==0:
            return False
    return True

def search_prime(values):
    
    for value in values:
        if is_prime(value):
            yield(value)

In [7]:
for prime in search_prime(values):
    print(prime, end=' ')

2 11 13 3 

In [11]:
import org

bosch= org.get_organization('Bosch',50)


### Search logic for permanent employees

In [7]:
def search_permanent_employees(org):
    for employee in org:
        if employee._isPermanent:
            yield employee

In [10]:
count=0
for e in search_permanent_employees(bosch):
    count+=1
    
print(count)

32


### Problem

* we are writing multiple search functions
* They have largely common code
* they differ just at point

### Approach #2

In [15]:
def search(values,criteria):
    results=[]
    for value in values:
        match=False
        if criteria=='evens':
            match= value%2==0
        elif criteria=='prime':
            match= is_prime(value)
        elif criteria=='permanent_employees':
            match= value._isPermanent
        
        if match:
            results.append(value)

    return results

In [16]:
def print_results(results):
    for result in results:
        print(result, end=' ')

In [18]:
values=[2,9,11,4,15,18,7,61,8]
print_results(search(values,'evens'))

2 4 18 8 

In [19]:
print_results(search(values,'prime'))

2 11 7 61 

In [21]:
result= search(bosch,'permanent_employees')
count=0
for emp in result:
    count+=1
    print(emp._name, emp._isPermanent )

print('total ',count)

Employee3 True
Employee5 True
Employee6 True
Employee9 True
Employee10 True
Employee12 True
Employee13 True
Employee15 True
Employee16 True
Employee17 True
Employee18 True
Employee20 True
Employee22 True
Employee24 True
Employee25 True
Employee28 True
Employee29 True
Employee30 True
Employee32 True
Employee33 True
Employee34 True
Employee35 True
Employee36 True
Employee37 True
Employee38 True
Employee39 True
Employee40 True
Employee41 True
Employee42 True
Employee43 True
Employee44 True
Employee46 True
Employee48 True
total  33


### problem

* just like calculator we have multiple logic merged in one function
* if tomorrow we need more criteria we will have to write logic for each of them in the search

* remember each crieria in iteself is a function
    * can be represented with a function object

#### Passing a callback function

In [23]:
def search(values, criteria):
    results=[]
    for value in values:
        if criteria(value):
            results.append(value)

    return results

In [24]:
primes = search(values, is_prime)

for prime in primes:
    print(prime, end=' ')

2 11 7 61 

### to perform even search we can just define is_even function

In [25]:
def is_even(value): return value%2==0

In [26]:
evens = search(values, is_even)
print_results(evens)

2 4 18 8 

### Or we can use lambda function to pass search criteria

In [27]:
permanent = search(bosch, lambda emp: emp._isPermanent)
for emp in permanent:
    print(emp._name, end='\t')

Employee3	Employee5	Employee6	Employee9	Employee10	Employee12	Employee13	Employee15	Employee16	Employee17	Employee18	Employee20	Employee22	Employee24	Employee25	Employee28	Employee29	Employee30	Employee32	Employee33	Employee34	Employee35	Employee36	Employee37	Employee38	Employee39	Employee40	Employee41	Employee42	Employee43	Employee44	Employee46	Employee48	

### Now we can do different types of searches


#### Find all employess whose salary is less that 35000

In [31]:
result = search(bosch, lambda emp: emp._salary<35000)

for emp in result:
    print(emp._name, emp._salary)

Employee0 28000
Employee2 21000
Employee3 31000
Employee6 32000
Employee7 20000
Employee9 23000
Employee14 23000
Employee16 21000
Employee18 20000
Employee20 23000
Employee21 25000
Employee22 33000
Employee24 30000
Employee27 23000
Employee32 34000
Employee36 25000
Employee41 29000
Employee42 31000
Employee43 23000
Employee44 23000
Employee47 34000
Employee49 30000


### Function Returning Another Function

In [33]:
def plus(x,y): return x+y

def minus(x,y) : return x-y

def selector(opr):
    if opr=='plus':
        return  plus #return a function 
    elif opr=='minus':
        return minus
    else:
        raise ValueError('No such operator')

In [34]:
a= selector('plus')
b= selector('minus')
c= selector('plus')

In [36]:
print(a)
a(10,12)

<function plus at 0x00000197AA3A1260>


22

In [38]:
print(a is b) # False plus is not minus
print(a is c) # True plus is plus

False
True


In [40]:
print(a, id(a))
print(b, id(b))
print(c, id(c))

<function plus at 0x00000197AA3A1260> 1750907621984
<function minus at 0x00000197AA2F6B60> 1750906923872
<function plus at 0x00000197AA3A1260> 1750907621984


### call chain

In [42]:
r1=selector("plus")(2,3)
r2=selector("minus")(12,8)
print(r1,r2)

5 4


#### What we learnt.

* A function **selector** may return another function.
* each function has a id
* we can call the returned function.
* we can call both functions in one statement using call chains



## Part 2. INNER FUNCTION: We can define a function inside another function

* A function can return an inner function
* When a function is called it creates a new instance of inner function
* each call will return a different instance of inner function 
    * They may have same name and same use
        * but they will not be same function

In [43]:

def selector(opr):
    def plus(x,y): return x+y
    def minus(x,y) : return x-y
    
    if opr=='plus':
        return  plus #return a function 
    elif opr=='minus':
        return minus
    else:
        raise ValueError('No such operator')

### Here we we call selector("plus") twice, we get two different plus functions

In [44]:
x = selector('plus')
y = selector('plus')

# both function has same name and logic
print(x.__name__, y.__name__)
a,b=20,30
print(x(a,b), y(a,b))

plus plus
50 50


In [45]:
### But they are not same. They have different ids

print(id(x))
print(id(y))
print(x is y)

1750907623264
1750907627104
False


### What is the use of Having multiple copies of same function in memory?
* It uses a closure scope.

#### Standard scope for a functions local variable
* when we call a function, all the parameters and local variables are created
* when function completes all the parameter and local variabels are deleted.



In [47]:
class Variable:
    def __init__(self,name,value):
        self.name=name
        self.value=value
        print(f'variable {self.name} created with value = {self.value}')

    def __del__(self):
        print(f'variable {self.name} removed')

    def __str__(self):
        return f'{self.name} = {self.value}'


In [50]:
def call_me(param):
    local=Variable('local',10)
    print(param)
    print(local)

In [51]:
call_me(Variable('p',100))

variable p created with value = 100
variable local created with value = 10
p = 100
local = 10
variable p removed
variable local removed


### Closure Scope

* When a function returns an inner function, this approach can cause a problem
* consder the below code

In [52]:
def outer(p ):

    def inner():
        print(p)

    return inner

#### Challenge

* when we call 
```python
x = outer(Variable(10))
# outer function is over.
# if p is removed
# what will next line print
x()
```
* the function outer completes after returning the result
* As per standard rule, variable "p" will be removed from memory
* inner function is not called yet

* now when we try to call x() in calls inner(). inner() tries to print "p1"
    * but "p1" is remvoed. Right?


### Closure Scope

* If the outer function returns an inner, then all its parameter and locals will be available to the inner() function
* They will NOT be removed from memeory

In [57]:
def outer(p):
    local=Variable("local", p.value*2)
    def inner():
        print(local)
        print(p)


In [58]:
outer(Variable("p",100))

variable p created with value = 100
variable local created with value = 200
variable p removed
variable local removed


#### But if we return inner

In [59]:
def outer(p):
    local=Variable("local", p.value*2)
    def inner():
        print(local)
        print(p)

    return inner

In [60]:
a = outer(Variable("a",10))

variable a created with value = 10
variable local created with value = 20


In [61]:
b= outer(Variable('b',20))

variable b created with value = 20
variable local created with value = 40


##### Now a and b will remember their versionof local and parameter

In [63]:
a()

local = 20
a = 10


In [64]:
b()

local = 40
b = 20


#### How does this concept Help?

* we can pass additional functions to inner function by passing them to outer function
* how many parameter below function takes?


In [68]:
def multiplierOf(number):
    def multiply(value):
        return number*value
    return multiply

In [69]:
m19 = multiplierOf(19)
m17 = multiplierOf(17)

#### Note

* both m17 and m19 are same function multiply


In [70]:
print(m17.__name__)
print(m19.__name__)

multiply
multiply


### How man parameter is multiply function taking?
```python

def multiplierOf(number):
    def multiply(value):
        return number*value
    return multiply
```

* multiply function is officially taking just one parameter **value**
* but each instance will also remember another argument **number** from the closure
* Thus they are actually taking two parameters
* This could be very useful if you want to pass additional information to a function but can't do it officially

### Search Function revisited

* let us say we want to search a list of Books

In [2]:
class Book:
    def __init__(self,title,author,price):
        self.title=title
        self.author=author
        self.price=price

    def __str__(self):
        return f'{self.title} by {self.author} is {self.price}'
    

def print_books(books, caption=""):
    print(caption)
    print(f"{'Title'.center(20)}|{'Author'.center(30)}|{'Price'.center(10)}")
    print('-'*62)
    for book in books:
        print(f"{book.title.ljust(20)}|{book.author.ljust(30)}|{str(book.price).rjust(10)}")


def get_books():
    yield Book('The Accursed God','Vivek Dutta Mishra',399)
    yield Book('Rashmrathi','Ramdhari Singh Dinkar',199)
    yield Book('Kurukshetra','Ramdhari Singh Dinkar',99)
    yield Book('Manas','Vivek Dutta Mishra',299)
    yield Book('Kane and Abel','Jeffrey Archer',459)
    yield Book('Brethren','John Grisham',499)


    

In [3]:
books = get_books()
print_books(books)


       Title        |            Author            |  Price   
--------------------------------------------------------------
The Accursed God    |Vivek Dutta Mishra            |       399
Rashmrathi          |Ramdhari Singh Dinkar         |       199
Kurukshetra         |Ramdhari Singh Dinkar         |        99
Manas               |Vivek Dutta Mishra            |       299
Kane and Abel       |Jeffrey Archer                |       459
Brethren            |John Grisham                  |       499


### How do I search for books by 'Vivek'

* here we can call our search function

```python
def search(values, criteria):
    
    for value in values:
        if criteria(value):
            yield(value)

    
```

* How many parameter does criteria function take? 
    * only one parameter 
        * to check if the current object is match
    * search function passes this parameter

* How do I pass "Vivek" 
    * normally a match may need another parameter
        * you need something to match with

* What if I want to search all books in a price range min-max?
    * how do we pass two additional parameter, when criteria takes a single paramter


#### Option #1 Hard Code <--- **BAD IDEA**



In [4]:
def search(values, criteria):
    result=[]
    
    for value in values:
       
        if criteria(value):
            result.append(value)

    return result

In [5]:
def book_by_vivek(book):
    return book.author=='Vivek Dutta Mishra'

books=get_books()
result = search(books, book_by_vivek)

print_books(result)



       Title        |            Author            |  Price   
--------------------------------------------------------------
The Accursed God    |Vivek Dutta Mishra            |       399
Manas               |Vivek Dutta Mishra            |       299


#### Problem: We have to write similar function for each author

* books_by_archer
* books_by_grisham

* ideally author name should be a parameter


### Approach #2  Using Closure

* create criterial as an inner function
* pass additonal parameters to the outer function

* now we are passing two paramters
    * author to outer function
    * book to actual criteria defined as inner function

In [6]:
def by_author(author):
    
    def criteria(book):
        return author.lower() in book.author.lower()
    
    return criteria

In [7]:
books = get_books()

result = search(books, by_author('vivek'))

print_books(result)


       Title        |            Author            |  Price   
--------------------------------------------------------------
The Accursed God    |Vivek Dutta Mishra            |       399
Manas               |Vivek Dutta Mishra            |       299


In [8]:
books=get_books()

result = search(books, by_author('dinkar'))
print_books(result)


       Title        |            Author            |  Price   
--------------------------------------------------------------
Rashmrathi          |Ramdhari Singh Dinkar         |       199
Kurukshetra         |Ramdhari Singh Dinkar         |        99


#### Books in Price Range

In [9]:
def in_price_range(min,max=None):
    if max is None:
        min,max=0,min

    def criteria(book):
        return book.price>=min and book.price<=max
    return criteria

In [10]:
books=get_books()
result=search(books, in_price_range(300))
print_books(result)


       Title        |            Author            |  Price   
--------------------------------------------------------------
Rashmrathi          |Ramdhari Singh Dinkar         |       199
Kurukshetra         |Ramdhari Singh Dinkar         |        99
Manas               |Vivek Dutta Mishra            |       299


In [11]:
books=get_books()
result=search(books, in_price_range(300,1000))
print_books(result)


       Title        |            Author            |  Price   
--------------------------------------------------------------
The Accursed God    |Vivek Dutta Mishra            |       399
Kane and Abel       |Jeffrey Archer                |       459
Brethren            |John Grisham                  |       499


#### Assignment 2.4 Can there be a third approach to solve the above problem


### Approach #3 Lamdba Expression

* lambda can be used for some simpler use case where we need to pass few parameters


#### Lets find all books priced less than 300

In [12]:
books = [book for book in  get_books()]

In [13]:
result = search(books, lambda book: book.price<300)

print_books(result)


       Title        |            Author            |  Price   
--------------------------------------------------------------
Rashmrathi          |Ramdhari Singh Dinkar         |       199
Kurukshetra         |Ramdhari Singh Dinkar         |        99
Manas               |Vivek Dutta Mishra            |       299


#### Why not always do this?

* it may not be very readable in complex scenarios like search by author name
* will not work for multiple lines of logic.

In [15]:
#result = search(books, by_author('vivek'))
result = search(books, lambda book: "vivek" in book.author.lower())

print_books(result)


       Title        |            Author            |  Price   
--------------------------------------------------------------
The Accursed God    |Vivek Dutta Mishra            |       399
Manas               |Vivek Dutta Mishra            |       299


### Builtin sequence functions that works with callback

* there are several builtin algorithms that operate on iterable like list and others.

* common ones include
    * filter
        * works in the same way as our search
    * map
        * transforms and sequence of one type into another

* list has sort
    * to sort the list on some basis

### lets search using filter
* we can use the same callbacks as we used with our search

In [24]:
result1= filter(book_by_vivek, books)

print_books(result1)


       Title        |            Author            |  Price   
--------------------------------------------------------------
The Accursed God    |Vivek Dutta Mishra            |       399
Manas               |Vivek Dutta Mishra            |       299


In [25]:
result2 = filter(by_author('archer'),books)

print_books(result2)


       Title        |            Author            |  Price   
--------------------------------------------------------------
Kane and Abel       |Jeffrey Archer                |       459


In [26]:
result3 = filter(lambda b: b.price>300, books)
print_books(result3)


       Title        |            Author            |  Price   
--------------------------------------------------------------
The Accursed God    |Vivek Dutta Mishra            |       399
Kane and Abel       |Jeffrey Archer                |       459
Brethren            |John Grisham                  |       499


### map function

In [29]:
#return me the book titles for every book
result = map( lambda b: b.title.upper(), books)

for x in result:
    print(x)

THE ACCURSED GOD
RASHMRATHI
KURUKSHETRA
MANAS
KANE AND ABEL
BRETHREN


### sort

* sort is a list function to sort a list of values
* by default it sorts item in the natural ascending order

In [31]:
numbers= [2, 9, 11, 8 , 4, 15, 5]

# sort
numbers.sort()
print(numbers)

[2, 4, 5, 8, 9, 11, 15]


#### we may get a descending sort


In [33]:
numbers= [2, 9, 11, 8 , 4, 15, 5]

# sort
numbers.sort( reverse = True)
print(numbers)

[15, 11, 9, 8, 5, 4, 2]


### Worksing with user defined objects

* It can work with user defined object if it has \_\_lt\_\_ function available
    * x<y

* which is not available in our books currently

In [34]:
books.sort()

TypeError: '<' not supported between instances of 'Book' and 'Book'

### Let us adda **<** support

#### if we want to sort on price

In [35]:
Book.__lt__= lambda b1,b2: b1.price<b2.price

In [36]:
books.sort()

print_books(books)


       Title        |            Author            |  Price   
--------------------------------------------------------------
Kurukshetra         |Ramdhari Singh Dinkar         |        99
Rashmrathi          |Ramdhari Singh Dinkar         |       199
Manas               |Vivek Dutta Mishra            |       299
The Accursed God    |Vivek Dutta Mishra            |       399
Kane and Abel       |Jeffrey Archer                |       459
Brethren            |John Grisham                  |       499


#### But we may need to sort object on different criteria

* Sort on title
* Sort on Author
    * for same author sort on price

#### We can have only one **<** and many different criteria


### sort using callback

* sort takes a key which can help us pass the callback logic
    * it returns a key on which we want to sort

In [40]:


books.sort( key= lambda b: b.title)

print_books(books)


       Title        |            Author            |  Price   
--------------------------------------------------------------
Brethren            |John Grisham                  |       499
Kane and Abel       |Jeffrey Archer                |       459
Kurukshetra         |Ramdhari Singh Dinkar         |        99
Manas               |Vivek Dutta Mishra            |       299
Rashmrathi          |Ramdhari Singh Dinkar         |       199
The Accursed God    |Vivek Dutta Mishra            |       399


In [41]:
help(list.sort)

Help on method_descriptor:

sort(self, /, *, key=None, reverse=False)
    Sort the list in ascending order and return None.

    The sort is in-place (i.e. the list itself is modified) and stable (i.e. the
    order of two equal elements is maintained).

    If a key function is given, apply it once to each list item and sort them,
    ascending or descending, according to their function values.

    The reverse flag can be set to sort in descending order.

